In [8]:
import polars as pl

import nwec.utility_reporting.arrearages
import nwec.utils.excel
from nwec.constants import CLEAN_UTILITY_DATA, RAW_UTILITY_DATA

YEAR = 2023
NUM_MONTHS = 3
COLS_PER_MONTH = 5
SHEET_SEARCH_STRING = "Arrears"
ARREARAGE_SEARCH_STRING = "Past-due balances"
KLI_SEARCH_STRING = "known low-income"
spreadsheet = RAW_UTILITY_DATA / str(YEAR) / "pac.xlsx"

In [9]:
sheet_index = nwec.utils.excel.get_sheet_index_from_name(spreadsheet, SHEET_SEARCH_STRING)
df = pl.read_excel(spreadsheet, sheet_id=sheet_index, has_header=False)
arrearages = nwec.utility_reporting.arrearages.get_arrearages_df(
    df, NUM_MONTHS, COLS_PER_MONTH, ARREARAGE_SEARCH_STRING
)
kli_arrearages = nwec.utility_reporting.arrearages.get_arrearages_df(df, NUM_MONTHS, COLS_PER_MONTH, KLI_SEARCH_STRING)

# Arrearages

In [10]:
date_to_zip_offset = 1
source_date_format = "%Y%m"

arrearages = nwec.utility_reporting.arrearages.combine_arrearage_year_vintage_cols(
    arrearages, NUM_MONTHS, COLS_PER_MONTH, date_to_zip_offset, source_date_format
)

In [11]:
arrearages.sum()

Zip Code,10 2023 COUNT,10 2023 DAYS 31 60,10 2023 DAYS 61 90,10 2023 DAYS 91+,10 2023 ARREARS,11 2023 COUNT,11 2023 DAYS 31 60,11 2023 DAYS 61 90,11 2023 DAYS 91+,11 2023 ARREARS,12 2023 COUNT,12 2023 DAYS 31 60,12 2023 DAYS 61 90,12 2023 DAYS 91+,12 2023 ARREARS
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3566098,26654.0,2.6583e6,1.9116e6,6.8507e6,1.1421e7,26220.0,2.2677e6,1.5202e6,6.8535e6,1.0642e7,23822.0,2.6612e6,1.0868e6,6.7601e6,1.0508e7


# KLI Arrearages

In [12]:
date_to_zip_offset = 1
source_date_format = "%Y%m"

kli_arrearages = nwec.utility_reporting.arrearages.combine_arrearage_year_vintage_cols(
    kli_arrearages, NUM_MONTHS, COLS_PER_MONTH, date_to_zip_offset, source_date_format
)

In [13]:
kli_arrearages.sum()

Zip Code,10 2023 COUNT,10 2023 DAYS 31 60,10 2023 DAYS 61 90,10 2023 DAYS 91+,10 2023 ARREARS,11 2023 COUNT,11 2023 DAYS 31 60,11 2023 DAYS 61 90,11 2023 DAYS 91+,11 2023 ARREARS,12 2023 COUNT,12 2023 DAYS 31 60,12 2023 DAYS 61 90,12 2023 DAYS 91+,12 2023 ARREARS
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3170370,4753.0,411087.59,382778.86,1.6135e6,2.4073e6,4975.0,407395.37,323422.86,1.6590e6,2.3898e6,4773.0,506641.0,268497.22,1.6729e6,2.4481e6


# Save Output

In [14]:
arrearages = nwec.utility_reporting.arrearages.normalize_arrearage_cols(arrearages, NUM_MONTHS)
arrearages = arrearages.with_columns(pl.lit("PAC").alias("Utility"))
arrearages = arrearages.with_columns(pl.lit("Residential").alias("Customer Class"))

kli_arrearages = nwec.utility_reporting.arrearages.normalize_arrearage_cols(kli_arrearages, NUM_MONTHS)
kli_arrearages = kli_arrearages.with_columns(pl.lit("PAC").alias("Utility"))
kli_arrearages = kli_arrearages.with_columns(pl.lit("KLI").alias("Customer Class"))

pl.concat([arrearages, kli_arrearages]).write_ipc(CLEAN_UTILITY_DATA / "pac.arrow")